In [1]:
%load_ext autoreload
%autoreload 2

# XoT Tutorial

Import OpenAI model with langchain.

In [2]:
import os
from dotenv import load_dotenv

import langchain
from langchain_openai import AzureChatOpenAI, ChatOpenAI

from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path=".langchain_cache.db"))

langchain.debug = False  # set verbose mode to True

load_dotenv()

langchain_llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("HKUST_OPENAI_BASE"),
    api_key=os.getenv("HKUST_OPENAI_KEY"),
    api_version=os.getenv("HKUST_OPENAI_API_VERSION"),
    openai_api_type="azure",
    azure_deployment=os.getenv("HKUST_OPENAI_DEPLOYMENT_NAME"),
    verbose=True,
)

# Or use openai's model
langchain_llm = ChatOpenAI(
    base_url=os.getenv("VENDER_BASE_URL"),
    api_key=os.getenv("VENDER_API_KEY"),
    model="gpt-4o",
    verbose=True,
)

In [3]:
# a debug line to show you have the correct model setup
langchain_llm.invoke("What is LLM?")

AIMessage(content="LLM can refer to a few different things depending on the context:\n\n1. **Master of Laws (LL.M.)**: This is an advanced, postgraduate academic degree in law. Individuals who have already completed their initial law degree (such as a JD or LLB) often pursue an LL.M. to gain specialized knowledge in a particular area of law, such as international law, tax law, or human rights law.\n\n2. **Large Language Model**: In the context of artificial intelligence and machine learning, LLM refers to a type of neural network model that is trained on large amounts of text data to understand and generate human-like text. Examples include OpenAI's GPT-3, which is used in applications like language translation, text summarization, and conversational agents.\n\n3. **Landlord and Tenant Law (LLM)**: In some jurisdictions, LLM is used as an abbreviation for specific legal frameworks or courses related to landlord and tenant law.\n\nThe meaning of LLM will depend on the context in which i

## Direct Use of LLM

* Practice the use of LLM with langchain.
* Intent classifier: does the user want to have its tablet repaired?

In [4]:
from utils.intent_classifier import intent_classifier_prompt, intent_classifier_parser

intent_classifier_chain = (
    intent_classifier_prompt | langchain_llm | intent_classifier_parser
)

In [5]:
intent_classifier_chain.invoke(
    {"user_input": "My ipad cannot connect to bluetooth, I'm here to get some support."}
)

{'want_to_repair': 'Yes'}

## Chain of Thoughts Prompting

* Practice the use of CoT with langchain.
* Use warranty checker as an example.

In [6]:
from utils.warranty_module import warranty_parser, warranty_checker_prompt


warranty_checker_chain = warranty_checker_prompt | langchain_llm

In [7]:
output = warranty_checker_chain.invoke({"user_input": "I bought this item on May 1st, 2023."})
warranty_parser.invoke(output)

{'Analysis': "Let's think step by step. All products come with a 90-day warranty since purchase. Since the item was purchased on May 1st, 2023, the 90-day warranty would have expired on July 30th, 2023. Given today's date is May 20th, 2024, the standard warranty has expired. However, if an additional 2-year warranty was purchased, it would still be valid until May 1st, 2025.",
 'Warranty': 'Unsure',
 'Utterance': 'Did you purchase additional warranty?'}

## Tree of thoughts

* We use tree of thoughts to practice repair plan suggestion. Available plans are repair, replace and trade-in options. 
* 

In [8]:
from utils.repair_strategy import repair_strategy_prompt, repair_parser
from utils.replace_strategy import replacement_strategy_prompt, replacement_parser
from utils.tradein_strategy import tradein_strategy_prompt, tradein_parser

repair_strategy_chain = repair_strategy_prompt | langchain_llm
replacement_strategy_chain = replacement_strategy_prompt | langchain_llm
tradein_strategy_chain = tradein_strategy_prompt | langchain_llm

In [9]:
repair_strategy_chain.invoke(
    {"user_input": "I want to repair my device. The screen and the battery are broken."}
)
repair_parser.invoke(
    repair_strategy_chain.invoke(
        {
            "user_input": "I want to repair my device. The screen and the battery are broken."
        }
    )
)

{'Analysis': "Let's think step by step. The cost for fixing the screen is 120 USD, and the cost for fixing the battery is 30 USD. The total cost is 150 USD.",
 'Cost': '150 USD',
 'Utterance': 'The total cost to fix the product is 150 USD.'}

In [10]:
tradein_strategy_chain.invoke(
    {"user_input": "I want to trade in my device. The keyboard is not working"}
)
tradein_parser.invoke(
    tradein_strategy_chain.invoke(
        {"user_input": "I want to trade in my device. The keyboard is not working"}
    )
)

{'Analysis': 'The user has a minor issue with the keyboard not working. The user will get a 100 USD coupon.',
 'Coupon': '100 USD',
 'Utterance': 'You can get a 100 USD coupon if you want to trade in your old device for a new one.'}

In [11]:
replacement_strategy_chain.invoke({"user_input": "The screen is broken, I want to replace it.", "additional_info": "No warranty"})
replacement_parser.invoke(replacement_strategy_chain.invoke({"user_input": "The screen is broken, I want to replace it.", "additional_info": "No warranty"}))

{'Analysis': 'The user does not have a warranty, so they cannot get a replacement.',
 'Valid': 'No',
 'Utterance': "I'm sorry, but you cannot get a replacement. Only products with a warranty can be replaced."}

In [12]:
from utils.plan_selection import plan_selection_prompt, plan_parser

plan_evaluater_chain = plan_selection_prompt | langchain_llm

In [13]:
plan_evaluater_chain.invoke(
    {
        "replace_plan": "I'm sorry, but you cannot get a replacement. Only products with warranty can be replaced.",
        "repair_plan": "The total cost to fix the product is 150 USD.",
        "tradein_plan": "You can get a 100 USD coupon if you want to trade in your old device for a new one.",
        "user_preference": "None",
        "selected_plan": "repair",
        "ai_analysis": "Let's thing step by step. The cost for a new device is 249 USD, the user can get a coupon so he will spend 149 USD for the new device. The cost for fixing the old device is 150 USD. Therefore, it is better to buy a new device as the cost is almost the same.",
        "ai_response": "Maybe you should consider buying a new device instead of fixing the old one. The cost for fixing the old device is 150 USD, and the cost for a new device is 249 USD. You can get a 100 USD coupon if you want to trade in your old device for a new one. The total cost to fix the product is 150 USD.",
    }
)
plan_parser.invoke(
    plan_evaluater_chain.invoke(
        {
            "replace_plan": "Your device can be replaced.",
            "repair_plan": "The total cost to fix the product is 150 USD.",
            "tradein_plan": "You can get a 100 USD coupon if you want to trade in your old device for a new one.",
            "user_preference": "None",
        }
    )
)

{'Analysis': 'Your device can be replaced without any additional charge. However, if you prefer to keep your current device, the repair cost is 150 USD. Alternatively, you can get a 100 USD coupon if you want to trade in your old device for a new one, which costs 249 USD. This would make the trade-in option a cost of 149 USD.',
 'Cost': 'Free',
 'Plan': 'Replace',
 'Utterance': 'Your device is eligible for a free replacement. If you prefer to keep your current device, the repair cost is 150 USD. Alternatively, you can trade in your old device for a new one and receive a 100 USD coupon, making the total cost for a new device 149 USD. Please let us know which option you would prefer.'}

In [14]:
from langchain_core.runnables import RunnableParallel

# The tot structure
repair_strategy_chain = repair_strategy_prompt | langchain_llm | repair_parser
replacement_strategy_chain = (
    replacement_strategy_prompt | langchain_llm | replacement_parser
)
tradein_strategy_chain = tradein_strategy_prompt | langchain_llm | tradein_parser

tot_generater = RunnableParallel(repair_plan = repair_strategy_chain, replace_plan = replacement_strategy_chain, tradein_plan = tradein_strategy_chain)

plan_output = tot_generater.invoke({
    "user_input": "The screen is broken, I want to replace it.",
    "additional_info": "No warranty"
})

tot_scorer = plan_selection_prompt | langchain_llm | plan_parser

tot_scorer.invoke({
    "replace_plan": plan_output["replace_plan"]["Utterance"],
    "repair_plan": plan_output["repair_plan"]["Utterance"],
    "tradein_plan": plan_output["tradein_plan"]["Utterance"],
    "user_preference": "None",
})

{'Analysis': "Let's think step by step. The product cannot be replaced without additional charge. For repair, the cost for fixing the old device is 120 USD. For Trade-in, the cost for a new device is 249 USD, and the user can get a 50 USD coupon. So the user will spend 199 USD for the new device. Therefore, it is better to repair the old device as it is more cost-effective.",
 'Cost': '120 USD',
 'Plan': 'Repair',
 'Utterance': "It would be more cost-effective to repair your current device. The cost for fixing it is 120 USD, whereas if you opt for a trade-in, you would still need to spend 199 USD for a new device after applying the 50 USD coupon. Let us know if you'd like to proceed with the repair."}

In [15]:
## GoT structure
### Break down the repair plan
